In [12]:
# I'm going to do a KNN analysis of the postal codes areas in toronto. 
# I'm going to be able to determine, given a post code what are the best other districts to visit for food

In [1]:
import pandas as pd
import requests
import json
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors
import math
import numpy as np

In [14]:
df = pd.read_csv(r"C:\Users\BenjaminWeller\Documents\Projects\Notebook\Geospatial_Coordinates.csv")
df.rename(columns={"Postal Code":"Postcode"}, inplace=True)
df.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [20]:
def nearby_venues(names, latitudes, longitudes, radius=500, limit=1000):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)         
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']["id"],
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Postcode', 
                  'Postcode Latitude', 
                  'Postcode Longitude', 
                  'Venue',
                  "Venue_ID",
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']    
    return(nearby_venues)

In [21]:
toronto_venues = nearby_venues(names=df['Postcode'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

In [22]:
toronto_venues.to_excel(r"C:\Users\BenjaminWeller\Documents\Projects\Notebook\toronto_venues.xlsx")

In [4]:
toronto_venues = pd.read_excel(r"C:\Users\BenjaminWeller\Documents\Projects\Notebook\toronto_venues.xlsx")

In [3]:
toronto_venues.head()

,Unnamed: 0,Postcode,Postcode Latitude,Postcode Longitude,Venue,Venue_ID,Venue Latitude,Venue Longitude,Venue Category
0,0,M1B,43.806686,-79.194353,Wendy's,4bb6b9446edc76b0d771311c,43.807448,-79.199056,Fast Food Restaurant
1,1,M1B,43.806686,-79.194353,Interprovincial Group,5539e7d2498edaf4b02673ca,43.805630,-79.200378,Print Shop
2,2,M1C,43.784535,-79.160497,Chris Effects Painting,587eee906d349d5759059742,43.784343,-79.163742,Construction & Landscaping
3,3,M1C,43.784535,-79.160497,Royal Canadian Legion,4c23d3aaf7ced13a5ed7216d,43.782533,-79.163085,Bar
4,4,M1E,43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,4b6074e3f964a5200fe729e3,43.767697,-79.189914,Pizza Place


In [3]:
def get_likes(venue_ids):
    my_list = []
    for venue, number in zip(venue_ids, range(925)):
        #Get the likes for the place 
        url = f"https://api.foursquare.com/v2/venues/{venue}?&client_id={CLIENT_ID}&client_secret={CLIENT_SECRET}&v={VERSION}"
        json = requests.get(url).json()
        if json.get("response") and \
        json.get("response").get('venue') and \
        json.get("response").get('venue').get("likes") and \
        json.get("response").get('venue').get("likes").get("count"):
            likes = json.get("response").get('venue').get("likes").get("count")
            my_list.append([venue, likes])
        else:
            my_list.append([venue, None])
        retruned = pd.DataFrame(my_list)
        retruned.columns = ["Venue_ID","Likes"]
    return retruned

In [5]:
partial_venues = get_likes(toronto_venues["Venue_ID"])

In [6]:
partial_venues.shape

(925, 2)

In [7]:
partial_venues.head()

,Venue_ID,Likes
0,4bb6b9446edc76b0d771311c,1.0
1,5539e7d2498edaf4b02673ca,NaN
2,587eee906d349d5759059742,NaN
3,4c23d3aaf7ced13a5ed7216d,1.0
4,4b6074e3f964a5200fe729e3,9.0


In [13]:
partial_venues[partial_venues["Likes"] != None]

,Venue_ID,Likes
0,4bb6b9446edc76b0d771311c,None
1,5539e7d2498edaf4b02673ca,None
2,587eee906d349d5759059742,None
3,4c23d3aaf7ced13a5ed7216d,None
4,4b6074e3f964a5200fe729e3,None
5,4c62f34bde1b2d7fec89e370,None
6,522deb21abdf65cfbab70655,None
7,5411f741498e9ebd5e35d8bd,None
8,5b8004ff9fca56002cffd9cc,None
9,4c1c7f9bb306c9288f0464b7,None


In [8]:
partial_venues.to_excel(r"C:\Users\BenjaminWeller\Documents\Projects\Notebook\partial_venues.xlsx")

In [5]:
partial_venues = pd.read_excel(r"C:\Users\BenjaminWeller\Documents\Projects\Notebook\partial_venues.xlsx")

In [ ]:
partial_venues = partial_venues[partial_venues["1"]]

In [ ]:
toronto_venues = getNearbyVenues(names=new_df['Postcode'],
                                   latitudes=new_df['Latitude'],
                                   longitudes=new_df['Longitude']
                                  )

In [23]:
yolo_df = get_likes(toronto_venues["Venue_ID"])

KeyError: 'venue'

In [ ]:
yolo_df